In [ ]:
#20250514 Wed, jeri
"""
출력 방식 변경: 조회수, 업로드일
추가한 내용: 상품 조회, 영상의 더보기 설명란
"""
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import json
import time
import os
import pandas as pd
from datetime import datetime
# 버전 관리를 위해 import
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 옵션 설정
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--start-maximized') # 유튜브 더보기 클릭 및 정보 가져오기

# 크롬 드라이버 실행(자동 버전 관리)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 드라이버가 실행될 영상 url
video_url = 'https://www.youtube.com/watch?v=w7uwgwj6aZM'
driver.get(video_url)

# 페이지 로딩 대기
time.sleep(5)

# 오늘 날짜 문자열 (YYMMDD)
today_str = datetime.today().strftime('%y%m%d')

# "더보기" 버튼 클릭
try:
    more_button = driver.find_element(By.CSS_SELECTOR, 'tp-yt-paper-button#expand')
    more_button.click()
except:
    print("더보기 버튼을 찾을 수 없습니다.")

# 설명란 텍스트 추출
try:
    description = driver.find_element(By.ID, 'description-inline-expander').text
except:
    print("설명란을 찾을 수 없습니다.")


"""
채널명, 영상 제목, 구독자 수
"""
# 영상에서 들고온 정보들을 담을 딕셔너리
base_info_results = {}

base_info = {
    "channel": '//*[@id="text"]/a',
    "title": '//*[@id="title"]/h1',
    "subscriber": '//*[@id="owner-sub-count"]',
}

for bases, xpath in base_info.items():
    try:
        # 로딩될 때까지 최대 60초 기다림
        element = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )
        # bases에 들어간 title, channel 등등의 내용(text)을 "base_info_results" 딕셔너리에 추가
        base_info_results[bases] = element.text
    # error 처리
    except Exception as e:
        print(f"Error fetching {bases}: {e}")
        base_info_results[bases] = "Not found"


# 유튜브 링크, 추출일 컬럼 추가
base_info_results["youtube_link"] = 'https://www.youtube.com/watch?v=w7uwgwj6aZM'
base_info_results['추출일'] = today_str
base_info_results['더보기란 설명'] = description

# 컬럼명 매핑
column_name_mapping = {
    "channel": "채널명",
    "title": "영상 제목",
    "youtube_link": "영상 링크",
    "subscriber": "구독자 수",
}

# 원하는 컬럼 순서를 리스트로 지정
column_order = ['추출일', '채널명', '영상 제목', '영상 링크', '구독자 수', '더보기란 설명']

# DataFrame 생성 및 컬럼명 변경
df = pd.DataFrame([base_info_results])
df = df.rename(columns=column_name_mapping)

# 원하는 컬럼 순서로 재정렬
df = df[column_order]


# base info 출력
print("\n=== Base Info ===")
for key, value in base_info_results.items():
    print(f"{column_name_mapping.get(key, key)}: {value}")
# 더보기란 출력
print(description)


# 기본 파일명 설정
base_filename = f'침착맨_유튜브_{today_str}'
ext = '.xlsx'
filename = f'{base_filename}{ext}'

# 엑셀로 저장
df.to_excel(filename, index=False)
print(f"\n✅ 엑셀 파일로 저장 완료: {filename}")


# 드라이버 종료
try:
    # scraping logic here
    pass
finally:
    driver.quit()




=== Base Info ===
채널명: 침착맨
영상 제목: 버거킹, 우리의 아이디어를 꼭 들어주시기 바랍니다
구독자 수: 구독자 281만명
영상 링크: https://www.youtube.com/watch?v=w7uwgwj6aZM
추출일: 250514
더보기란 설명: 이 영상은 [버거킹]의 유료 광고가 포함되어 있습니다.

▶같이 보면 좋은 추천 영상
   • 담 걸린 침착맨의 아이언맨 맛 버거 먹방  
▶'침착맨의 일상재롱' 모아보기
   • 침착맨의 일상재롱  

▶이 영상의 생방송 원본
  •2025년 4월 28일 방송분:    • 2025년 04월 28일 1부 | 버거킹 신메뉴 크리스퍼 2종 먹방  

▶출연: 침착맨
▶편집: 15수자
▶썸네일: 15수자 

▶협업제안: info@chimchakman.com

#침착맨 #버거킹 #크리스퍼 #크리스퍼클래식 #크리스퍼클래식BLT #치킨버거 #IDONTLIKEWHOPPER #ILOVEKRISPPER #노윤서 #추영우
음악
노래 1곡
Feel The Funk
Jimmy Fontanez/Media Right Productions
Feel The Funk
음악
스크립트
스크립트를 보면서 시청하세요.
스크립트 표시
침착맨
구독자 281만명
동영상
정보
토크 영상은? 침착맨
게임 영상은? 침착맨 플러스
생방송은? 침착맨 원본 박물관
침착맨 공식 인스타그램
간략히

=== More info button ===
이 영상은 [버거킹]의 유료 광고가 포함되어 있습니다.

▶같이 보면 좋은 추천 영상
   • 담 걸린 침착맨의 아이언맨 맛 버거 먹방  
▶'침착맨의 일상재롱' 모아보기
   • 침착맨의 일상재롱  

▶이 영상의 생방송 원본
  •2025년 4월 28일 방송분:    • 2025년 04월 28일 1부 | 버거킹 신메뉴 크리스퍼 2종 먹방  

▶출연: 침착맨
▶편집: 15수자
▶썸네일: 15수자 

▶협업제안: info@chimchakman.com

#침착맨 #버거킹 #크리스퍼 #크리스퍼클래식 #크리스퍼클래